In [1]:
import pandas as pd
df = pd.DataFrame()
import csv

In [2]:
#Create csv file
jan1_csv = open('jan1.csv', 'w')
#Column names
col_name = ['building', 'date', 'month', 'time', 'device_count']
csv_writer = csv.DictWriter(jan1_csv, fieldnames=col_name)
csv_writer.writeheader()


39

In [3]:
def processLine(buildingName='UREC', wifi_log=''):
    count = 0
    for log_line in wifi_log:
        if buildingName in log_line:
            # process device count
            if '501080' or '501105' or '501106' in log_line:
                count -= 0
            elif '501093' in log_line:
                count += 1
            # Slice the line to get the date and time
            atr = log_line.split(' ')
            # Extract the data
            month = atr[0]
            date = atr[2]
            time = atr[3]
            building = buildingName
            # add data to csv file
            csv_writer.writerow({'building': building, 'date': date, 'month': month, 'time': time, 'device_count': count})


In [4]:
wifi_log = open('wireless_01-01-2021.log', 'r')
# # print first ten lines in the file
# for i in range(3):
#     print(wifi_log.readline())

# loop through each line in the file
processLine(wifi_log=wifi_log)

# close the file
wifi_log.close()


In [5]:
# convert jan1.csv to dataframe
df = pd.read_csv('jan1.csv')
# pandes datafram dispaly all rows
pd.set_option('display.max_rows', None)
df
# # length of dataframe
# len(df)

,building,date,month,time,device_count
0,UREC,1,Jan,00:01:29,0
1,UREC,1,Jan,00:01:29,0
2,UREC,1,Jan,00:01:29,0
3,UREC,1,Jan,00:01:29,0
4,UREC,1,Jan,00:01:38,0
5,UREC,1,Jan,00:01:38,0
6,UREC,1,Jan,00:01:38,0
7,UREC,1,Jan,00:01:38,0
8,UREC,1,Jan,00:01:38,0
9,UREC,1,Jan,00:01:38,0
